In [1]:
import pandas as pd
import numpy as np
import itertools

# Visualización de grafos
import networkx as nx
from networkx.algorithms import bipartite
from pyvis.network import Network

# Interacciones en el notebook
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
# Load Data
artworks_data = pd.read_csv('/data/Prado_artworks_wikidata.csv')
interactions = pd.read_csv('/data/Prado_emotions.csv')

# Lista de usuarios para los nodos
users_nodes = np.unique(interactions.userId.values)

# Creo diccionarios para obtener imagenes y títulos de los cuadros
artwork_ids = np.unique(interactions.artworkId.values)
artworks_dict = dict()
images_dict = dict()
for art_id in artwork_ids:
    title = artworks_data[artworks_data['ID'] == art_id]['Title'].values[0]
    image = artworks_data[artworks_data['ID'] == art_id]['Image URL'].values[0]
    artworks_dict[art_id] = title
    images_dict[title] = image
    
# Create array of emotions and its colors
colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080']
emotions = np.unique(interactions['emotion'].values)
emotions_colors = dict()
for i in range(len(emotions)):
    emotions_colors[emotions[i]] = colors[i]
    
emotions_options = list(emotions)
emotions_options.append(None)

In [3]:
# Creamos el grafo bipartito
G = nx.Graph()

# Incluyo los nodos
G.add_nodes_from(users_nodes, bipartite=0)
G.add_nodes_from(list(artworks_dict.values()), bipartite=1)

# Incluyo los ejes
for i in range(len(interactions)):
    user = interactions.iloc[i]['userId']
    artwork = artworks_dict[interactions.iloc[i]['artworkId']]
    emotion = interactions.iloc[i]['emotion']
    polarity = interactions.iloc[i]['Polarity']
    G.add_edge(user, artwork, color=emotions_colors[emotion], emotion=emotion, polarity=polarity) 

In [4]:
@interact_manual
def visualize_graph_by_emotion(emotion=emotions_options, show_physics=False):
    vis = Network(notebook=True, width="1000px", height="800px")
    
    for node, node_attr in G.nodes(data=True):
        if node_attr['bipartite'] == 0:
            vis.add_node(str(node), **node_attr, group=node_attr['bipartite'], size=1)
        else:
            vis.add_node(str(node), **node_attr, group=node_attr['bipartite'], size=30, shape='image', image =images_dict[node])
    
    for source,target,edge_attrs in G.edges(data=True):
        if emotion != None:
            if edge_attrs['emotion'] == emotion:
                vis.add_edge(str(source), str(target), **edge_attrs)
        else:
            vis.add_edge(str(source), str(target), **edge_attrs)
    
    if show_physics:
        vis.show_buttons(filter_=['physics'])
    
    return vis.show('graph_bipartite.html')

interactive(children=(Dropdown(description='emotion', options=('agreeableness', 'anger', 'anticipation', 'arro…